In [9]:
import nglview as nv
import numpy as np
import pytraj as pt 
import matplotlib.pyplot as plt

from hilbertcurve.hilbertcurve import HilbertCurve


from matplotlib import cm


In [17]:
class DummyPDB:
    def __init__(self):
        self.pdbline = ""
        self.temp = "ATOM      1  Du  TMP     1       0.000   0.000   0.000  1.00  0.00";
    
    def addCoor(self, vecX, vecY, vecZ, elems):
        newtxt = self.convertToPDB(vecX, vecY, vecZ, elems); 
        self.pdbline += newtxt;
        
    def convertToPDB(self, vecX, vecY, vecZ, elems):
        if (len(vecX) != len(vecY)) or (len(vecX) != len(vecZ)):
            print("Lengths of each dimension are not equil")
        pdbline = "";
        
        for i in range(len(vecX)):
            tmpcopy=self.temp;
            indlen = len(str(i));
            atmInd = "{:>5}".format(i+1);
            X = "{:>8}".format(vecX[i]);
            Y = "{:>8}".format(vecY[i]);
            Z = "{:>8}".format(vecZ[i]);
            try:
                Elem = elems[i];
            except:
                Elem = "Du";
            Elem = "{:>4}".format(Elem);
            tmpcopy = tmpcopy[:12]+Elem+tmpcopy[16:]; 
            tmpcopy = tmpcopy[:6]+atmInd+tmpcopy[11:]; 
            tmpcopy = tmpcopy[:30]+X+tmpcopy[38:]; 
            tmpcopy = tmpcopy[:38]+Y+tmpcopy[46:]; 
            tmpcopy = tmpcopy[:46]+Z+tmpcopy[54:];
            pdbline += tmpcopy+"\n"
        
        return pdbline
    
    def writeFile(self, outname;):
        self.outname = outname;
        with open(outname, 'w') as outfile:
            outfile.write(self.pdbline)
            
    def show(self):
        viewer = nglview.show_file(self.outname)
        return viewer

    
a=DummyPDB()
b=a.convertToPDB([1.003,2.34],[2.125,13.2],[3.516,2.34],["CL","CL"])
print(b)

ATOM      1   CL TMP     1       1.003   2.125   3.516  1.00  0.00
ATOM      2   CL TMP     1        2.34    13.2    2.34  1.00  0.00



In [91]:
import re
from numpy import rot90  # FOR rotation of the matrix

class hilbertize:
    def __init__(self, iterNr, dimNr):
        self.iterNr = iterNr;
        self.dimNr = dimNr; 
        self.len3D = 2 ** self.iterNr;   # 16 for hilbert(4,3)
        self.len2D = np.sqrt(2**(self.iterNr*self.dimNr))   # 64 for hilbert(4,3)
        self.distances = np.array(range(2**(self.iterNr*self.dimNr)));
        self.curve = HilbertCurve(self.iterNr, self.dimNr); 
        self.points = self.curve.points_from_distances(self.distances); 
    
    def from_mol2(self, filename):
        with open(filename,"r") as file2:
            self.mol2file = [i.strip('\n') for i in file2.readlines()]
        inatom = False
        self.mol2x=[];self.mol2y=[];self.mol2z=[];
        for i in self.mol2file:
            if ("<TRIPOS>ATOM" in i):
                inatom = True;
                continue
            elif ("<TRIPOS>BOND" in i):
                inatom = False
            if (inatom):
                self.mol2x.append(float(i.split()[2]))
                self.mol2y.append(float(i.split()[3]))
                self.mol2z.append(float(i.split()[4]))
        self.center = [np.mean(self.mol2x), np.mean(self.mol2y),np.mean(self.mol2z)]
        
    def from_pdb(self, filename):
        with open(filename,"r") as file1:
            self.pdbfile = [i.strip('\n') for i in  file1.readlines()] 
        try: 
#             raise
            self.trajpdb = pt.load(filename)
            self.trajpdb.top.set_reference(self.trajpdb[0])
            self.pdbx = self.trajpdb.xyz[0][:,0]
            self.pdby = self.trajpdb.xyz[0][:,1]
            self.pdbz = self.trajpdb.xyz[0][:,2]
            try:
                self.center = [i for i in pt.center_of_geometry(self.trajpdb)[0]]
            except:
                self.center = [np.mean(self.pdbx), np.mean(self.pdby), np.mean(self.pdbz)]
        except:
            self.pdbx = []; self.pdby = []; self.pdbz = []; 
            for i in self.pdbfile:
                if ("ATOM" in i) or ("HETATM" in i):
                    self.pdbx.append(float(i[30:38].strip())); 
                    self.pdby.append(float(i[38:46].strip())); 
                    self.pdbz.append(float(i[46:54].strip())); 
            self.pdbx = np.array(self.pdbx); 
            self.pdby = np.array(self.pdby); 
            self.pdbz = np.array(self.pdbz); 
            self.center = [np.mean(self.pdbx), np.mean(self.pdby),np.mean(self.pdbz)]
        
        try: 
            AtomLst = [i.name for i in self.trajpdb.top.atoms]
            c=0;
            for i in self.pdbfile:
                if ("ATOM" in i) or ("HETATM" in i):
                    self.elems.append(AtomLst[c])
                    c += 1
        except:
            self.elems = []; 
            for i in self.pdbfile:
                if ("ATOM" in i) or ("HETATM" in i):
                    self.elems.append(i[12:16].strip())
        self.elems = np.array(self.elems)
    def align(self, refCenter):
        diff = np.array(this.center) - np.array(refCenter); 
        self.points = self.points + diff
    
    def transform(self, shift, refLength):
        LenSc = refLength / self.len3D
    
    def shift(self, shift):
        self.points = self.points + np.array(shift)
    
    def reduceDim(self):
        pass
    
    def T(self, direction):   
        # up, down, left, right
        # two up, two down, two left, two right
        pass
    
    
    def assignElem(self, cutoff):
#         def assignElem(self, cutoff, Hpoints, trajObj, refcutoff):
        # Read the coordinate and the center of mass. 
        c1=0; c2=0; c3=0
        try: 
            masklst = []
            xyzlst = []
            for i in range(len(self.pdbx)):
                tmpcoor = np.array([ self.pdbx[i], self.pdby[i], self.pdbz[i] ]);
                centerDiff = np.sqrt(np.sum((tmpcoor - self.center)**2))
                if (centerDiff <= cutoff):
                    masklst.append(True)
                    xyzlst.append(tmpcoor)
                else: 
                    masklst.append(False)
            masklst = np.array(masklst)
            xyzlst = np.array(xyzlst)
            elemlst = self.elems[masklst]
        except:  
            maskStr = ":LIG <@ "+str(cutoff);
            self.trajpdb.top.set_reference(self.trajpdb[0])
            tmpselect = self.trajpdb.top.select(maskStr)
            elemlst = np.array([i.name for i in self.trajpdb.top.atoms])[tmpselect]
            xyzlst = self.trajpdb.xyz[0][tmpselect]
        self.elemdistance = []
        for i in range(len(self.points)):
            diff = self.points[i] - xyzlst;
            sums = np.sum(np.square(diff), axis = 1); 
            distmin = np.round(np.sqrt(np.min(sums)),4)
            if (distmin < 2.5): #0.866):
                c1 += 1
                self.elemdistance.append(elemlst[sums == np.min(sums)][0])
            elif (distmin > 1.732):
                c2 += 1
                self.elemdistance.append("Du")
            else :
                c3 += 1
                self.elemdistance.append("Du")
#         print( self.elemdistance)
#         print(c1, c2, c3)

    def show():
        tmpline = DummyPDB();
        tmpline.addCoor(self.pdbx, self.pdby, self.pdbz, self.elemdistance)


HBC_lig = hilbertize(4,3); 
HBC_pro = hilbertize(4,3); 
HBC_lig.from_mol2("trajLigand.mol2")
HBC_pro.from_pdb("rec.pdb")
HBC_pro.assignElem(12)
print(HBC_pro.pdbx, HBC_pro.center)

[37.28  38.647 39.517 ...  0.802 -0.396  0.756] [9.127052617003583, -3.9814458598726, -7.443824979230133]


In [ ]:
# [37.28  38.647 39.517 ...  0.802 -0.396  0.756] [9.1270526170036, -3.9814458598726112, -7.443824979230131]

In [26]:



tmp=traj.top.atoms
select1 = traj.top.select(":LIG <@ 12 & !:LIG, T3P & !@CL,K ")
tmpelems = np.array([i.name for i in traj.top.atoms])[select1]
tmpxyz = traj.xyz[0][select1]
tmpx = tmpxyz[:,0]
tmpy = tmpxyz[:,1]
tmpz = tmpxyz[:,2]

points1 = np.array(points1)
points2 = np.array(points2)

x2 = points2[:,0]
y2 = points2[:,1] 

pointT = (points1) + (COMLIG-8)


c1=0; c2=0; c3=0
print("points", len(pointT))
print("tmpxyz", len(tmpxyz))

dum2elems = []
for i in range(len(pointT)):
    diff = pointT[i] - tmpxyz;
    sums = np.sum(np.square(diff), axis = 1); 
    distmin = np.round(np.sqrt(np.min(sums)),4)
    if (distmin < 2.5):   #0.866):
        c1 += 1
        dum2elems.append(tmpelems[sums == np.min(sums)][0])
    elif (distmin > 1.732):
        c2 += 1
        dum2elems.append("Du")
    else :
        c3 += 1
        dum2elems.append("Du")
print(c1, c2, c3)



HCx = np.round(pointT[:,0], 4)
HCy = np.round(pointT[:,1], 4)
HCz = np.round(pointT[:,2], 4)


dummypdb1 = DummyPDB(outname="tmp1.pdb")
dummypdb2 = DummyPDB(outname="tmp2.pdb")
line1 = dummypdb1.convertToPDB(HCx,HCy,HCz, dum2elems)
line2 = dummypdb2.convertToPDB(tmpx, tmpy, tmpz, tmpelems)
print(line2)


points 4096
tmpxyz 744
3717 379 0
ATOM      1  HB1 TMP     1       3.147   2.004  -6.778  1.00  0.00
ATOM      2  HB2 TMP     1       3.665   0.461  -7.371  1.00  0.00
ATOM      3   CA TMP     1       6.852   3.264  -5.041  1.00  0.00
ATOM      4    C TMP     1       6.044   2.723  -3.871  1.00  0.00
ATOM      5    O TMP     1       4.885   3.105  -3.677  1.00  0.00
ATOM      6   HG TMP     1       7.272   4.419  -2.647  1.00  0.00
ATOM      7 HD23 TMP     1       7.094   6.582  -2.094  1.00  0.00
ATOM      8    N TMP     1       6.632   1.795  -3.131  1.00  0.00
ATOM      9   CA TMP     1       6.068   1.297  -1.882  1.00  0.00
ATOM     10    C TMP     1       6.945   1.809  -0.747  1.00  0.00
ATOM     11    O TMP     1       8.131   1.467  -0.679  1.00  0.00
ATOM     12   HN TMP     1       7.521   1.418   -3.46  1.00  0.00
ATOM     13   HA TMP     1       5.066     1.7  -1.738  1.00  0.00
ATOM     14   CB TMP     1       6.004  -0.233  -1.866  1.00  0.00
ATOM     15  CG1 TMP     1  

In [73]:
traj1 = pt.load("rec.pdb")

nv.show_pytraj(traj1)
# print([i.name for i in traj1.top.atoms])
dir(traj1[0])
print(type(traj1[0]))

<class 'pytraj.trajectory.frame.Frame'>


In [ ]:
traj = pt.load("rec.pdb")


print(traj.xyz[0])

traj.top.set_reference(traj[0])

proXYZ = traj.xyz[0][traj.top.select(":1-183")]
ovcenter = np.mean(traj.xyz[0][traj.top.select(":LIG <: 5 & !:LIG")], axis=0)
ligcenter = np.mean(traj.xyz[0][traj.top.select(":LIG")], axis = 0)

atomSel = traj.top.select(":LIG <: 5 & !:LIG")
COMLIG = pt.center_of_mass(traj=traj, mask=atomSel)[0]
print(ovcenter)
print(ligcenter)
print(COMLIG)